In [1]:
# lstmで日経平均の上下の予測(多変量)

In [2]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/kaggle/RNN

/content/drive/MyDrive/kaggle/RNN


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [5]:
# データの読み込み
train1 = pd.read_csv("data/N225_2003.csv")
train2 = pd.read_csv("data/NYダウ平均株価2003.csv")
train3 = pd.read_csv("data/USD_JPY2003.csv")

In [6]:
print(train1.head())
print(train2.head())
print(train3.head())

           日付け         終値         始値         高値         安値      出来高    前日比%
0  2021年12月30日  28,791.71  28,794.24  28,904.42  28,579.49  403.89M  -0.40%
1  2021年12月29日  28,906.88  28,995.73  29,106.28  28,729.61  447.11M  -0.56%
2  2021年12月28日  29,069.16  28,953.32  29,121.01  28,879.68  470.40M   1.37%
3  2021年12月27日  28,676.46  28,786.33  28,805.28  28,658.82  374.87M  -0.37%
4  2021年12月24日  28,782.59  28,836.05  28,870.13  28,773.50  359.09M  -0.05%
           日付け         終値         始値         高値         安値      出来高    前日比%
0  2021年12月31日  36,338.30  36,385.85  36,484.92  36,304.53  205.29M  -0.16%
1  2021年12月30日  36,398.08  36,558.11  36,679.09  36,372.13  197.77M  -0.25%
2  2021年12月29日  36,488.63  36,421.14  36,569.85  36,398.51  206.45M   0.25%
3  2021年12月28日  36,398.67  36,322.51  36,526.89  36,318.73  231.82M   0.26%
4  2021年12月27日  36,302.58  36,023.79  36,304.17  35,985.01  241.55M   0.98%
           日付け      終値      始値      高値      安値    前日比%
0  2022年01月02日  115.14  115.12  1

In [7]:
# 日経平均上下
train1['前日比%'] = train1['前日比%'].str.replace('%', '')
train1['日経上下'] = train1['前日比%'].astype(float)

for k in range(len(train1)):
  if train1['日経上下'][k] < 0:
    train1['日経上下'][k] = 0
  else:
    train1['日経上下'][k] = 1

print(train1.head())

# NYダウ平均上下
train2['前日比%'] = train2['前日比%'].str.replace('%', '')
train2['ダウ上下'] = train2['前日比%'].astype(float)

for k in range(len(train2)):
  if train2['ダウ上下'][k] < 0:
    train2['ダウ上下'][k] = 0
  else:
    train2['ダウ上下'][k] = 1

print(train2.head())

# ドル円上下
train3['前日比%'] = train3['前日比%'].str.replace('%', '')
train3['ドル円上下'] = train3['前日比%'].astype(float)

for k in range(len(train3)):
  if train3['ドル円上下'][k] < 0:
    train3['ドル円上下'][k] = 0
  else:
    train3['ドル円上下'][k] = 1

print(train3.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


           日付け         終値         始値  ...      出来高   前日比% 日経上下
0  2021年12月30日  28,791.71  28,794.24  ...  403.89M  -0.40  0.0
1  2021年12月29日  28,906.88  28,995.73  ...  447.11M  -0.56  0.0
2  2021年12月28日  29,069.16  28,953.32  ...  470.40M   1.37  1.0
3  2021年12月27日  28,676.46  28,786.33  ...  374.87M  -0.37  0.0
4  2021年12月24日  28,782.59  28,836.05  ...  359.09M  -0.05  0.0

[5 rows x 8 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


           日付け         終値         始値  ...      出来高   前日比% ダウ上下
0  2021年12月31日  36,338.30  36,385.85  ...  205.29M  -0.16  0.0
1  2021年12月30日  36,398.08  36,558.11  ...  197.77M  -0.25  0.0
2  2021年12月29日  36,488.63  36,421.14  ...  206.45M   0.25  1.0
3  2021年12月28日  36,398.67  36,322.51  ...  231.82M   0.26  1.0
4  2021年12月27日  36,302.58  36,023.79  ...  241.55M   0.98  1.0

[5 rows x 8 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


           日付け      終値      始値      高値      安値   前日比%  ドル円上下
0  2022年01月02日  115.14  115.12  115.15  115.07   0.06    1.0
1  2021年12月31日  115.08  115.07  115.20  115.00   0.02    1.0
2  2021年12月30日  115.06  114.96  115.21  114.93   0.10    1.0
3  2021年12月29日  114.94  114.82  115.04  114.67   0.11    1.0
4  2021年12月28日  114.81  114.87  114.96  114.71  -0.02    0.0


In [8]:
# データの型をobjectからfloatに変換
# その前にカンマを取り除く
#train['終値_x'] = pd.to_numeric(train['終値_x'], errors = 'coerce')
train1['終値'] = train1['終値'].str.replace(',', '')
train2['終値'] = train2['終値'].str.replace(',', '')
train1['終値'] = train1['終値'].astype(float)
train2['終値'] = train2['終値'].astype(float)
print(train1.head(), train2.head())
print(train1.info(), train2.info())

           日付け        終値         始値         高値         安値      出来高   前日比%  日経上下
0  2021年12月30日  28791.71  28,794.24  28,904.42  28,579.49  403.89M  -0.40   0.0
1  2021年12月29日  28906.88  28,995.73  29,106.28  28,729.61  447.11M  -0.56   0.0
2  2021年12月28日  29069.16  28,953.32  29,121.01  28,879.68  470.40M   1.37   1.0
3  2021年12月27日  28676.46  28,786.33  28,805.28  28,658.82  374.87M  -0.37   0.0
4  2021年12月24日  28782.59  28,836.05  28,870.13  28,773.50  359.09M  -0.05   0.0            日付け        終値         始値         高値         安値      出来高   前日比%  ダウ上下
0  2021年12月31日  36338.30  36,385.85  36,484.92  36,304.53  205.29M  -0.16   0.0
1  2021年12月30日  36398.08  36,558.11  36,679.09  36,372.13  197.77M  -0.25   0.0
2  2021年12月29日  36488.63  36,421.14  36,569.85  36,398.51  206.45M   0.25   1.0
3  2021年12月28日  36398.67  36,322.51  36,526.89  36,318.73  231.82M   0.26   1.0
4  2021年12月27日  36302.58  36,023.79  36,304.17  35,985.01  241.55M   0.98   1.0
<class 'pandas.core.frame.DataFrame'>
Ra

In [9]:
# データフレームを結合
train = pd.merge(train1, train2, on='日付け', how='left')
train = pd.merge(train, train3, on='日付け', how='left')
train = train.dropna(how='any')
print(train.head(), train.isnull().sum())

           日付け      終値_x       始値_x       高値_x  ...      高値      安値   前日比%  ドル円上下
0  2021年12月30日  28791.71  28,794.24  28,904.42  ...  115.21  114.93   0.10    1.0
1  2021年12月29日  28906.88  28,995.73  29,106.28  ...  115.04  114.67   0.11    1.0
2  2021年12月28日  29069.16  28,953.32  29,121.01  ...  114.96  114.71  -0.02    0.0
3  2021年12月27日  28676.46  28,786.33  28,805.28  ...  114.93  114.30   0.37    1.0
5  2021年12月23日  28798.37  28,703.01  28,798.37  ...  114.48  114.07   0.23    1.0

[5 rows x 21 columns] 日付け       0
終値_x      0
始値_x      0
高値_x      0
安値_x      0
出来高_x     0
前日比%_x    0
日経上下      0
終値_y      0
始値_y      0
高値_y      0
安値_y      0
出来高_y     0
前日比%_y    0
ダウ上下      0
終値        0
始値        0
高値        0
安値        0
前日比%      0
ドル円上下     0
dtype: int64


In [10]:
train = train[['終値_x', '日経上下', '終値_y', 'ダウ上下', '終値', 'ドル円上下']]
train

,終値_x,日経上下,終値_y,ダウ上下,終値,ドル円上下
0,28791.71,0.0,36398.08,0.0,115.06,1.0
1,28906.88,0.0,36488.63,1.0,114.94,1.0
2,29069.16,1.0,36398.67,1.0,114.81,0.0
3,28676.46,0.0,36302.58,1.0,114.83,1.0
5,28798.37,1.0,35950.63,1.0,114.37,1.0
...,...,...,...,...,...,...
4675,8470.45,0.0,8784.89,1.0,119.28,0.0
4676,8497.93,0.0,8776.18,1.0,119.32,1.0
4677,8517.80,0.0,8595.31,0.0,119.06,0.0
4678,8656.50,0.0,8740.59,0.0,120.43,1.0


In [11]:
train = train.reset_index(drop=True)
train

,終値_x,日経上下,終値_y,ダウ上下,終値,ドル円上下
0,28791.71,0.0,36398.08,0.0,115.06,1.0
1,28906.88,0.0,36488.63,1.0,114.94,1.0
2,29069.16,1.0,36398.67,1.0,114.81,0.0
3,28676.46,0.0,36302.58,1.0,114.83,1.0
4,28798.37,1.0,35950.63,1.0,114.37,1.0
...,...,...,...,...,...,...
4495,8470.45,0.0,8784.89,1.0,119.28,0.0
4496,8497.93,0.0,8776.18,1.0,119.32,1.0
4497,8517.80,0.0,8595.31,0.0,119.06,0.0
4498,8656.50,0.0,8740.59,0.0,120.43,1.0


In [12]:
train.head(100)

,終値_x,日経上下,終値_y,ダウ上下,終値,ドル円上下
0,28791.71,0.0,36398.08,0.0,115.06,1.0
1,28906.88,0.0,36488.63,1.0,114.94,1.0
2,29069.16,1.0,36398.67,1.0,114.81,0.0
3,28676.46,0.0,36302.58,1.0,114.83,1.0
4,28798.37,1.0,35950.63,1.0,114.37,1.0
...,...,...,...,...,...,...
95,27888.15,1.0,35264.67,1.0,110.56,1.0
96,27820.04,1.0,35208.51,1.0,110.25,1.0
97,27728.12,1.0,35064.25,1.0,109.74,1.0
98,27584.08,0.0,34792.67,0.0,109.47,1.0


In [13]:
# 標準化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train)
train_scaled = scaler.transform(train)
print('train_scaled = \n', train_scaled[0:5], len(train_scaled))

train1_reshaped = train['終値_x'].values.reshape(-1, 1)
scaler1 = StandardScaler()
scaler1.fit(train1_reshaped)
train1_scaled = scaler1.transform(train1_reshaped)
train['終値_x'] = train1_scaled
print('train1_scaled = \n', train1_scaled[0:5], len(train1_scaled))

train2_reshaped = train['終値_y'].values.reshape(-1, 1)
scaler2 = StandardScaler()
scaler2.fit(train2_reshaped)
train2_scaled = scaler2.transform(train2_reshaped)
train['終値_y'] = train2_scaled
print('train2_scaled = \n', train2_scaled[0:5], len(train2_scaled))

train3_reshaped = train['終値'].values.reshape(-1, 1)
scaler3 = StandardScaler()
scaler3.fit(train3_reshaped)
train3_scaled = scaler3.transform(train3_reshaped)
train['終値'] = train3_scaled
print('train3_scaled = \n', train3_scaled[0:5], len(train3_scaled))

train_scaled = 
 [[ 2.33144728 -1.04687527  2.81713904 -1.09174826  0.82391473  0.98063197]
 [ 2.35184409 -1.04687527  2.82988982  0.91596207  0.81411507  0.98063197]
 [ 2.38058417  0.95522363  2.81722212  0.91596207  0.80349877 -1.01975056]
 [ 2.31103631 -1.04687527  2.80369122  0.91596207  0.80513204  0.98063197]
 [ 2.33262678  0.95522363  2.75413142  0.91596207  0.76756667  0.98063197]] 4500
train1_scaled = 
 [[2.33144728]
 [2.35184409]
 [2.38058417]
 [2.31103631]
 [2.33262678]] 4500
train2_scaled = 
 [[2.81713904]
 [2.82988982]
 [2.81722212]
 [2.80369122]
 [2.75413142]] 4500
train3_scaled = 
 [[0.82391473]
 [0.81411507]
 [0.80349877]
 [0.80513204]
 [0.76756667]] 4500


In [14]:
train

,終値_x,日経上下,終値_y,ダウ上下,終値,ドル円上下
0,2.331447,0.0,2.817139,0.0,0.823915,1.0
1,2.351844,0.0,2.829890,1.0,0.814115,1.0
2,2.380584,1.0,2.817222,1.0,0.803499,0.0
3,2.311036,0.0,2.803691,1.0,0.805132,1.0
4,2.332627,1.0,2.754131,1.0,0.767567,1.0
...,...,...,...,...,...,...
4495,-1.267484,0.0,-1.071209,1.0,1.168536,0.0
4496,-1.262617,0.0,-1.072435,1.0,1.171803,1.0
4497,-1.259098,0.0,-1.097904,0.0,1.150570,0.0
4498,-1.234534,0.0,-1.077447,0.0,1.262450,1.0


In [15]:
# 訓練データとテストデータに分割

# それぞれを標準化
x_train = train[0:4200]
x_test = train[4200:4450]
print('x_train = \n', x_train[0:5], len(x_train))
print('x_test = \n', x_test[0:5], len(x_test))

# 一緒に標準化
x2_train = train_scaled[0:4200]
x2_test = train_scaled[4200:4450]
print('x2_train = \n', x2_train[0:5], len(x2_train))
print('x2_test = \n', x2_test[0:5], len(x2_test))

# 単変量で上下予測用
x0_train = train['日経上下'][0:4200]
x0_test = train['日経上下'][4200:4450]
print('x0_train = \n', x0_train[0:5], len(x0_train))
print('x0_test = \n', x0_test[0:5], len(x0_test))

# 3つの上下データだけ
x3_train = train[['日経上下', 'ダウ上下', 'ドル円上下']][0:4200]
x3_test =  train[['日経上下', 'ダウ上下', 'ドル円上下']][4200:4450]
print('x3_train = \n', x3_train[0:5], len(x3_train))
print('x3_test = \n', x3_test[0:5], len(x3_test))

# 正解データ
y_train = train['日経上下'][51:4201]
y_test = train['日経上下'][4251:4451]
print('y_train = \n', y_train[0:5], len(y_train))
print('y_test = \n', y_test[0:5], len(y_test))

x_train = 
        終値_x  日経上下      終値_y  ダウ上下        終値  ドル円上下
0  2.331447   0.0  2.817139   0.0  0.823915    1.0
1  2.351844   0.0  2.829890   1.0  0.814115    1.0
2  2.380584   1.0  2.817222   1.0  0.803499    0.0
3  2.311036   0.0  2.803691   1.0  0.805132    1.0
4  2.332627   1.0  2.754131   1.0  0.767567    1.0 4200
x_test = 
           終値_x  日経上下      終値_y  ダウ上下        終値  ドル円上下
4200 -0.628279   1.0 -0.819725   1.0  0.063624    1.0
4201 -0.649776   1.0 -0.821476   1.0 -0.000890    1.0
4202 -0.674990   1.0 -0.833837   1.0 -0.036822    1.0
4203 -0.698461   0.0 -0.847533   1.0 -0.102154    0.0
4204 -0.692799   1.0 -0.849734   0.0 -0.054788    0.0 250
x2_train = 
 [[ 2.33144728 -1.04687527  2.81713904 -1.09174826  0.82391473  0.98063197]
 [ 2.35184409 -1.04687527  2.82988982  0.91596207  0.81411507  0.98063197]
 [ 2.38058417  0.95522363  2.81722212  0.91596207  0.80349877 -1.01975056]
 [ 2.31103631 -1.04687527  2.80369122  0.91596207  0.80513204  0.98063197]
 [ 2.33262678  0.95522363

In [16]:
# ５０行１セットで訓練データを作成
length = 50
x_train_data = []
x_test_data = []
x2_train_data = []
x2_test_data = []
x0_train_data = []
x0_test_data = []
x3_train_data = []
x3_test_data = []

y_data = [] #予測したいデータ

for i in range(len(x_train)-length):
    x_train_data.append(x_train[i:i+50])
    x2_train_data.append(x2_train[i:i+50])
    x0_train_data.append(x0_train[i:i+50])
    x3_train_data.append(x3_train[i:i+50])

for i in range(len(x_test)-length):
    x_test_data.append(x_test[i:i+50])
    x2_test_data.append(x2_test[i:i+50])
    x0_test_data.append(x0_test[i:i+50])
    x3_test_data.append(x3_test[i:i+50])

x_train = np.asarray(x_train_data)
x_test = np.asarray(x_test_data)
x2_train = np.asarray(x2_train_data)
x2_test = np.asarray(x2_test_data)
x0_train = np.asarray(x0_train_data)
x0_test = np.asarray(x0_test_data)
x3_train = np.asarray(x3_train_data)
x3_test = np.asarray(x3_test_data)

#x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
#x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

#y_train = y_train.reshape(y_train.shape[0], 1)
#y_test = y_test.reshape(y_test.shape[0], 1)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

#x0_train = tf.convert_to_tensor(x0_train)
#x0_test = tf.convert_to_tensor(x0_test)

(4150, 50, 6)
(200, 50, 6)
(4150,)
(200,)


In [17]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation  

'''
# 多変量lstmで予測
data_dim = 6
length_of_sequences = 50
#batch_size = 10250
h_num = 50
in_out_neurons = 1

model = Sequential()  
#model.add(LSTM(h_num, batch_input_shape=(None, length_of_sequences, in_out_neurons), return_sequences=False))
model.add(LSTM(h_num, input_shape=(50, 6), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(20))
model.add(Dense(in_out_neurons))
model.add(Activation("sigmoid"))
model.compile(loss="mean_squared_error", optimizer="adam") # optimizer="rmsprop"
model.fit(x_train, y_train, batch_size=100, epochs=100)
'''

'\n# 多変量lstmで予測\ndata_dim = 6\nlength_of_sequences = 50\n#batch_size = 10250\nh_num = 50\nin_out_neurons = 1\n\nmodel = Sequential()  \n#model.add(LSTM(h_num, batch_input_shape=(None, length_of_sequences, in_out_neurons), return_sequences=False))\nmodel.add(LSTM(h_num, input_shape=(50, 6), return_sequences=False))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(20))\nmodel.add(Dense(in_out_neurons))\nmodel.add(Activation("sigmoid"))\nmodel.compile(loss="mean_squared_error", optimizer="adam") # optimizer="rmsprop"\nmodel.fit(x_train, y_train, batch_size=100, epochs=100)\n'

In [18]:
'''
y_submit = []
y_submit = model.predict(x_test, batch_size=100, verbose=1)
print(y_submit.shape)
print(y_submit[0:5])
for k in range(len(y_submit)):
  if y_submit[k] > 0.5:
    y_submit[k] = 1
  else:
    y_submit[k] = 0
accuracy = 0
for s in range(len(y_submit)):
  if y_submit[s] == y_test[s+4251]:
    accuracy += 1
accuracy = accuracy/len(y_submit)*100
print('accuracy =', accuracy, '%')
'''

"\ny_submit = []\ny_submit = model.predict(x_test, batch_size=100, verbose=1)\nprint(y_submit.shape)\nprint(y_submit[0:5])\nfor k in range(len(y_submit)):\n  if y_submit[k] > 0.5:\n    y_submit[k] = 1\n  else:\n    y_submit[k] = 0\naccuracy = 0\nfor s in range(len(y_submit)):\n  if y_submit[s] == y_test[s+4251]:\n    accuracy += 1\naccuracy = accuracy/len(y_submit)*100\nprint('accuracy =', accuracy, '%')\n"

In [19]:
# 単変量で上下予測
data_dim = 1
length_of_sequences = 50
batch_size = len(x0_train)
h_num = 100
in_out_neurons = 1

model0 = Sequential()  
model0.add(LSTM(h_num, input_shape=(length_of_sequences, in_out_neurons), return_sequences=False))
model0.add(Dropout(0.3))
model0.add(Dense(50))
model0.add(Dense(20))
model0.add(Dense(in_out_neurons))
model0.add(Activation('sigmoid'))
model0.compile(loss="mean_squared_error", optimizer="adam") #  optimizer="rmsprop"
model0.fit(x0_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 5s 7ms/step - loss: 0.2509
Epoch 2/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2503
Epoch 3/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2496
Epoch 4/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2505
Epoch 5/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2497
Epoch 6/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2497
Epoch 7/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2497
Epoch 8/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2498
Epoch 9/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2498
Epoch 10/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2499
Epoch 11/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2499
Epoch 12/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2498
Epoch 13/100
42/42 [=================

In [20]:
y0_submit = []
y0_submit = model0.predict(x0_test, batch_size=100, verbose=1)
print(y0_submit.shape)
print(y0_submit[0:5])

for k in range(len(y0_submit)):
  if y0_submit[k] > 0.5:
    y0_submit[k] = 1
  else:
    y0_submit[k] = 0
accuracy0 = 0
for s in range(len(y0_submit)):
  if y0_submit[s] == y_test[s+4251]:
    accuracy0 += 1
accuracy0 = accuracy0/len(y0_submit)*100
print('accuracy0 =', accuracy0, '%')

2/2 [==============================] - 0s 7ms/step
(200, 1)
[[0.5264086 ]
 [0.528599  ]
 [0.52155423]
 [0.5190385 ]
 [0.52288353]]
accuracy0 = 58.5 %


In [21]:
# 一緒に標準化
data_dim = 6
length_of_sequences = 50
batch_size = len(x0_train)
h_num = 100
in_out_neurons = 1

model2 = Sequential()  
model2.add(LSTM(h_num, input_shape=(50, 6), return_sequences=False))
model2.add(Dropout(0.3))
model2.add(Dense(20))
model2.add(Dense(in_out_neurons))
model2.add(Activation("sigmoid"))
model2.compile(loss="mean_squared_error", optimizer="rmsprop") #  optimizer="rmsprop"
model2.fit(x2_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 2s 8ms/step - loss: 0.2511
Epoch 2/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2501
Epoch 3/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2500
Epoch 4/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2499
Epoch 5/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2497
Epoch 6/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2488
Epoch 7/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2495
Epoch 8/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2495
Epoch 9/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2484
Epoch 10/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2485
Epoch 11/100
42/42 [==============================] - 0s 7ms/step - loss: 0.2487
Epoch 12/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2479
Epoch 13/100
42/42 [=================

In [22]:
y2_submit = []
y2_submit = model2.predict(x2_test, batch_size=100, verbose=1)
print(y2_submit.shape)
print(y2_submit[0:5])

for k in range(len(y2_submit)):
  if y2_submit[k] > 0.5:
    y2_submit[k] = 1
  else:
    y2_submit[k] = 0
accuracy2 = 0
for s in range(len(y2_submit)):
  if y2_submit[s] == y_test[s+4251]:
    accuracy2 += 1
accuracy2 = accuracy2/len(y2_submit)*100
print('accuracy2 =', accuracy2, '%')

2/2 [==============================] - 0s 6ms/step
(200, 1)
[[0.82737356]
 [0.5680233 ]
 [0.7319158 ]
 [0.83650464]
 [0.537867  ]]
accuracy2 = 48.0 %


In [23]:
data_dim = 3
length_of_sequences = 50
batch_size = len(x3_train)
h_num = 100
in_out_neurons = 1

model3 = Sequential()  
model3.add(LSTM(h_num, input_shape=(50, 3), return_sequences=False))
model3.add(Dropout(0.3))
model3.add(Dense(20))
model3.add(Dense(in_out_neurons))
model3.add(Activation("sigmoid"))
model3.compile(loss="mean_squared_error", optimizer="adam") #  optimizer="rmsprop"
model3.fit(x3_train, y_train, batch_size=100, epochs=100)

Epoch 1/100
42/42 [==============================] - 2s 8ms/step - loss: 0.2507
Epoch 2/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2502
Epoch 3/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2499
Epoch 4/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2496
Epoch 5/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2494
Epoch 6/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2496
Epoch 7/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2496
Epoch 8/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2495
Epoch 9/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2494
Epoch 10/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2488
Epoch 11/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2493
Epoch 12/100
42/42 [==============================] - 0s 6ms/step - loss: 0.2495
Epoch 13/100
42/42 [=================

In [24]:
y3_submit = []
y3_submit = model3.predict(x3_test, batch_size=100, verbose=1)
print(y3_submit.shape)
print(y3_submit[0:5])

for k in range(len(y3_submit)):
  if y3_submit[k] > 0.5:
    y3_submit[k] = 1
  else:
    y3_submit[k] = 0
accuracy3 = 0
for s in range(len(y3_submit)):
  if y3_submit[s] == y_test[s+4251]:
    accuracy3 += 1
accuracy3 = accuracy3/len(y3_submit)*100
print('accuracy3 =', accuracy3, '%')

2/2 [==============================] - 0s 8ms/step
(200, 1)
[[0.5459422 ]
 [0.30763033]
 [0.6469765 ]
 [0.73011297]
 [0.7996879 ]]
accuracy3 = 49.0 %
